In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
import warnings
import pickle
import time
import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import ToktokTokenizer,word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import hamming_loss
from sklearn.cluster import KMeans

import logging
from scipy.sparse import hstack
warnings.filterwarnings("ignore")
plt.style.use('bmh')
%matplotlib inline

In [6]:
#nltk.download()

# Read Data

In [2]:

Question_df=pd.read_csv("/kaggle/input/stacksample/Questions.csv",encoding="ISO-8859-1")
Answer_df=pd.read_csv("/kaggle/input/stacksample/Answers.csv",encoding="ISO-8859-1")
Tag_df=pd.read_csv("/kaggle/input/stacksample/Tags.csv",encoding="ISO-8859-1")



In [3]:
Question_df.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [4]:
Question_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264216 entries, 0 to 1264215
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Id            1264216 non-null  int64  
 1   OwnerUserId   1249762 non-null  float64
 2   CreationDate  1264216 non-null  object 
 3   ClosedDate    55959 non-null    object 
 4   Score         1264216 non-null  int64  
 5   Title         1264216 non-null  object 
 6   Body          1264216 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 67.5+ MB


In [8]:
Answer_df.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...
3,269,91.0,2008-08-01T23:49:57Z,260,4,"<p>Yes, I thought about that, but I soon figur..."
4,307,49.0,2008-08-02T01:49:46Z,260,28,"<p><a href=""http://www.codeproject.com/Article..."


In [9]:
Answer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2014516 entries, 0 to 2014515
Data columns (total 6 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Id            int64  
 1   OwnerUserId   float64
 2   CreationDate  object 
 3   ParentId      int64  
 4   Score         int64  
 5   Body          object 
dtypes: float64(1), int64(3), object(2)
memory usage: 92.2+ MB


In [5]:
Tag_df.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [6]:
Tag_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750994 entries, 0 to 3750993
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Id      int64 
 1   Tag     object
dtypes: int64(1), object(1)
memory usage: 57.2+ MB


In [7]:
print(Question_df.shape,Tag_df.shape,Answer_df.shape)

(1264216, 7) (3750994, 2) (2014516, 6)


In [8]:
Tag_df["Tag"]=Tag_df["Tag"].astype(str)

#### I'll have to group tags by the id of the post since a post can have multiple tags

In [9]:
Tags = Tag_df.groupby("Id")['Tag'].apply(lambda Tag_df: ' '.join(Tag_df))

In [10]:
Tags.head()

Id
80                            flex actionscript-3 air
90       svn tortoisesvn branch branching-and-merging
120                               sql asp.net sitemap
180    algorithm language-agnostic colors color-space
260           c# .net scripting compiler-construction
Name: Tag, dtype: object

In [11]:
Tags.reset_index()

,Id,Tag
0,80,flex actionscript-3 air
1,90,svn tortoisesvn branch branching-and-merging
2,120,sql asp.net sitemap
3,180,algorithm language-agnostic colors color-space
4,260,c# .net scripting compiler-construction
...,...,...
1264211,40143210,php .htaccess
1264212,40143300,google-bigquery
1264213,40143340,android android-studio
1264214,40143360,javascript vue.js


In [12]:
final_tags_df=pd.DataFrame({"Id":Tags.index,"Tags":Tags.values})


In [13]:
final_tags_df.head(3)

,Id,Tags
0,80,flex actionscript-3 air
1,90,svn tortoisesvn branch branching-and-merging
2,120,sql asp.net sitemap


#### Make preprocessing on data

In [14]:
#Remove unnecessry columns
Question_df.drop(columns=['OwnerUserId', 'CreationDate', 'ClosedDate'], inplace=True)

#### Merge both dataframe (question,tag)

In [15]:
final_df = Question_df.merge(final_tags_df, on='Id')


In [16]:
final_df

,Id,Score,Title,Body,Tags
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,flex actionscript-3 air
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,svn tortoisesvn branch branching-and-merging
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql asp.net sitemap
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,algorithm language-agnostic colors color-space
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,c# .net scripting compiler-construction
...,...,...,...,...,...
1264211,40143210,0,URL routing in PHP (MVC),<p>I am building a custom MVC project and I ha...,php .htaccess
1264212,40143300,0,Bigquery.Jobs.Insert - Resumable Upload?,<p>The API docs show that you should be able t...,google-bigquery
1264213,40143340,1,Obfuscating code in android studio,<p>Under minifyEnabled I changed from false to...,android android-studio
1264214,40143360,0,How to fire function after v-model change?,<p>I have input which I use to filter my array...,javascript vue.js


In [17]:
print(final_df["Score"].min())
print(final_df["Score"].max())

-73
5190


#### take only quesions witha score greater than 4,since:
* 1- I'll require less computational resources from kaggle.
* 2- The posts will probably be with a better quality and will be better tagged since they have lots of upvotes. 

In [18]:
final_df=final_df[final_df["Score"]>4]

In [19]:
final_df['Score'].count()

93153

In [20]:
final_df.drop(columns=["Id","Score"],inplace=True)

In [21]:
final_df

,Title,Body,Tags
0,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,flex actionscript-3 air
1,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,svn tortoisesvn branch branching-and-merging
2,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql asp.net sitemap
3,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,algorithm language-agnostic colors color-space
4,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,c# .net scripting compiler-construction
...,...,...,...
1263253,CMFCMenuButton not properly repainting when to...,"<p>In an C++ MFC project I'm using <a href=""ht...",c++ mfc accessibility high-contrast cmfcmenubu...
1263399,How can I force file ordering in F# projects u...,<p>I'm trying to work with vscode in my WebSha...,f# vscode
1263454,Why does my result data returned as void* gets...,<p>I am working in a project with a huge legac...,c++
1263609,Do I need to extend FirebaseInstanceIdService ...,<p>I want to manage topic subscription from th...,android firebase firebase-cloud-messaging


In [22]:
final_df.isnull().sum()

Title    0
Body     0
Tags     0
dtype: int64

In [23]:
final_df.duplicated().sum()

0

In [24]:
#there is 1 duplicated
final_df.drop_duplicates(inplace = True)

##### very good dataset since -> no missing valeus or dupplicate values.
##### we only need 3 columns: Body, Title and Tags. 

# Tags

#####  do cleaning on the tags' column and keep the 150 most popular tags because I'll be easier to predict the right tag from 150 words than from 286401 words.

In [26]:
# Split tags in order to get a list of tags
final_df['Tags'] = final_df['Tags'].apply(lambda x:x.split())
all_tags = [item for sublist in final_df['Tags'].values for item in sublist]

In [27]:
len(all_tags)

286401

In [28]:
final_df

,Title,Body,Tags
0,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[flex, actionscript-3, air]"
1,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"[svn, tortoisesvn, branch, branching-and-merging]"
2,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net, sitemap]"
3,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,"[algorithm, language-agnostic, colors, color-s..."
4,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net, scripting, compiler-construction]"
...,...,...,...
1263253,CMFCMenuButton not properly repainting when to...,"<p>In an C++ MFC project I'm using <a href=""ht...","[c++, mfc, accessibility, high-contrast, cmfcm..."
1263399,How can I force file ordering in F# projects u...,<p>I'm trying to work with vscode in my WebSha...,"[f#, vscode]"
1263454,Why does my result data returned as void* gets...,<p>I am working in a project with a huge legac...,[c++]
1263609,Do I need to extend FirebaseInstanceIdService ...,<p>I want to manage topic subscription from th...,"[android, firebase, firebase-cloud-messaging]"


In [29]:
#unique tags
m_set= set(all_tags)
unique_tags = list(m_set)
len(unique_tags)

16632

In [30]:
flatten_the_list = [item for sublist in final_df['Tags'].values for item in sublist]
words = nltk.FreqDist(flatten_the_list)
frequencies_words =words.most_common(25)
important_tags = [word[0] for word in frequencies_words]

In [31]:
important_tags

['c#',
 'java',
 'javascript',
 'android',
 'python',
 'c++',
 'php',
 'jquery',
 'ios',
 '.net',
 'html',
 'css',
 'c',
 'objective-c',
 'iphone',
 'ruby-on-rails',
 'sql',
 'asp.net',
 'mysql',
 'ruby',
 'r',
 'asp.net-mvc',
 'git',
 'linux',
 'sql-server',
 'angularjs',
 'node.js',
 'django',
 'algorithm',
 'performance',
 'arrays',
 'wpf',
 'string',
 'xcode',
 'eclipse',
 'windows',
 'scala',
 'haskell',
 'multithreading',
 'regex',
 'json',
 'c++11',
 'html5',
 'osx',
 'swift',
 'visual-studio',
 'database',
 'spring',
 'xml',
 'unit-testing',
 'bash',
 'ajax',
 'delphi',
 'linq',
 'ruby-on-rails-3',
 'visual-studio-2010',
 'winforms',
 'perl',
 'entity-framework',
 'generics',
 'cocoa-touch',
 'debugging',
 'mongodb',
 'cocoa',
 'security',
 'postgresql',
 'templates',
 'css3',
 'google-chrome',
 'oracle',
 'image',
 'list',
 'twitter-bootstrap',
 'oop',
 'hibernate',
 'facebook',
 'maven',
 'vim',
 'shell',
 'gcc',
 'asp.net-mvc-3',
 'rest',
 'numpy',
 'qt',
 'tsql',
 'exceptio

In [35]:
#this function check if tags in column of Tags_df exist in important_tags
def common_tags(tags):
    final_tags = []
    for i in range(0, len(tags)):
        if tags[i] in important_tags:
            final_tags.append(tags[i])
            break
    return final_tags

# if the tag exist in important_tags apply it else apply none
final_df['Tags'] =final_df['Tags'].apply(lambda x: common_tags(x))
final_df['Tags'] =final_df['Tags'].apply(lambda x: x if len(x)>0 else None)

# Drop rows that contain None in Tags column
final_df.dropna(subset=['Tags'], inplace=True)
final_df.shape

(80393, 3)

In [36]:
final_df.isnull().sum()

Title    0
Body     0
Tags     0
dtype: int64

In [37]:
final_df['Tags']

1                [svn]
2                [sql]
3          [algorithm]
4                 [c#]
5                [c++]
              ...     
1263065       [python]
1263253          [c++]
1263454          [c++]
1263609      [android]
1264205      [haskell]
Name: Tags, Length: 80393, dtype: object

In [38]:
final_df

,Title,Body,Tags
1,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,[svn]
2,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,[sql]
3,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,[algorithm]
4,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,[c#]
5,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,[c++]
...,...,...,...
1263065,Is there a way to use itertools in python to c...,<p>Let's say I have the following code:</p>\n\...,[python]
1263253,CMFCMenuButton not properly repainting when to...,"<p>In an C++ MFC project I'm using <a href=""ht...",[c++]
1263454,Why does my result data returned as void* gets...,<p>I am working in a project with a huge legac...,[c++]
1263609,Do I need to extend FirebaseInstanceIdService ...,<p>I want to manage topic subscription from th...,[android]


# Body & Title

* Removing html format 
* Lowering text
* Removing stop words
* Removing punctuation (but keeping words like c# since it's the most popular tag)
* Lemmatizing words
* stemming words


In [39]:
# Converting html to text in the body
final_df['Body'] = final_df['Body'].apply(lambda x: BeautifulSoup(x).get_text()) 

In [40]:
final_df.head(3)

,Title,Body,Tags
1,Good branching and merging tutorials for Torto...,Are there any really good tutorials explaining...,[svn]
2,ASP.NET Site Maps,Has anyone got experience creating SQL-based A...,[sql]
3,Function for creating color wheels,This is something I've pseudo-solved many time...,[algorithm]


In [41]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [42]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [43]:
punct='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [44]:
#this function used to remove any whitespace characters from the beginning and end of the string
def strip_list_noempty(mylist):
    newlist = (item.strip() if hasattr(item, 'strip') else item for item in mylist)
    return [item for item in newlist if item != '']

In [45]:
# this function to remove punctuation
token=ToktokTokenizer()
def remove_punct(text): 
    words=token.tokenize(text)
    punctuation_filtered = []
    regex = re.compile('[%s]' % re.escape(punct))
    remove_punctuation = str.maketrans(' ', ' ', punct)
    for w in words:
        if w in important_tags:
            punctuation_filtered.append(w)
        else:
            punctuation_filtered.append(regex.sub('', w))
  
    filtered_list = strip_list_noempty(punctuation_filtered)
        
    return ' '.join(map(str, filtered_list))

In [46]:
lemma=WordNetLemmatizer()
stemm=PorterStemmer()
stop_words = set(stopwords.words("english"))

In [47]:
# this function to apply lemmatization
def lemitize_Words(text):
    words=token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w, pos="v")
        listLemma.append(x)
    return ' '.join(map(str, listLemma))
# this function to apply stemming
def stemming_Words(text):
    words=token.tokenize(text)
    liststem=[]
    for w in words:
        x=stemm.stem(w)
        liststem.append(x)
    return ' '.join(map(str,liststem))
#this function to remove stopwords
def remove_stopwords(text):
    
    stop_words = set(stopwords.words("english"))
    
    words=token.tokenize(text)
    
    filtered = [w for w in words if not w in stop_words]
    
    return ' '.join(map(str, filtered))

#### apply all function on Body and Title columns

#### Body

In [48]:

final_df['Body'] = final_df['Body'].apply(lambda x: clean_text(x))  

In [49]:
final_df['Body'] = final_df['Body'].apply(lambda x: remove_punct(x)) 


In [50]:
final_df['Body'] = final_df['Body'].apply(lambda x: remove_stopwords(x))

In [51]:
final_df['Body'] = final_df['Body'].apply(lambda x: stemming_Words(x)) 

In [55]:
#there is problem in lemmatization in kaggle not in code , i will solve it with you, since i am tired, and i will use stemming 
#final_df['Body'] = final_df['Body'].apply(lambda x: lemitize_Words(x)) 

#### Title

In [52]:
#Title
final_df['Title'] = final_df['Title'].apply(lambda x: str(x))


In [53]:
final_df['Title'] = final_df['Title'].apply(lambda x: clean_text(x)) 
 

In [54]:
final_df['Title'] = final_df['Title'].apply(lambda x: remove_punct(x)) 


In [55]:
final_df['Title'] = final_df['Title'].apply(lambda x: remove_stopwords(x))

In [56]:
final_df['Title'] = final_df['Title'].apply(lambda x: stemming_Words(x))

In [ ]:
#final_df['Title'] = final_df['Title'].apply(lambda x: lemitize_Words(x)) 


In [57]:
final_df["Title"]

1                        good branch merg tutori tortoisesvn
2                                           asp.net site map
3                                 function creat color wheel
4                             ad script function .net applic
5                                        use nest class case
                                 ...                        
1263065              way use itertool python clean nest iter
1263253    cmfcmenubutton properli repaint toggl high con...
1263454                   result data return void get broken
1263609    need extend firebaseinstanceidservic subscrib ...
1264205                     exactli make type system consist
Name: Title, Length: 80393, dtype: object

In [60]:
final_df.to_csv("preprocessed Automatic Question Tagging System data.csv",index=False)